In [1]:
# 01_AssignGenes.py 
# Perform VDJ assignment with IgBLAST.
# This function use IgBLAST germline reference to assign VDJ sequences of my data. 
# -s for my input VDJ sequence
# -b for the reference database

import os

os.mkdir("/home/phil/wt/results_032622_pm")

!AssignGenes.py igblast -s /home/phil/wt/filtered_contig.fasta \
-b /usr/local/share/igblast/ --organism mouse --loci ig \
--format blast --nproc 8 --outdir /home/phil/wt/results_032622_pm/

# 02_MakeDb.py
# Converting IgBLAST format (fmt7) into AIRR community format
# for downstream analysis. 
# -s for my input VDJ sequence
# - i for inputing IgBLAST result (fmt7)
# -r for the IMGT reference database

!MakeDb.py igblast -s /home/phil/wt/filtered_contig.fasta \
-i /home/phil/wt/results_032622_pm/filtered_contig_igblast.fmt7 \
--format airr --10x /home/phil/wt/filtered_contig_annotations.csv \
-r /usr/local/share/germlines/imgt/mouse/vdj --outdir /home/phil/wt/results_032622_pm/

#03_ParseDb.py
# Including the productive H chain sequences
# -d for inputing AIRR format IGBLAST assigned VDJ germline sequences
# -f for selecting "productive" H chains
# -u for "TRUE" values

!ParseDb.py select -d /home/phil/wt/results_032622_pm/filtered_contig_igblast_db-pass.tsv \
-f productive -u T --outdir /home/phil/wt/results_032622_pm/

#04_ParseDb.py 
# Selecting only productive H chain sequences
# -d for inputting the previous file
# -f for speicifing the v_call column
# -u for selecting the H chain 
# --regex for pattern matching

!ParseDb.py select -d /home/phil/wt/results_032622_pm/filtered_contig_igblast_db-pass_parse-select.tsv \
-f v_call -u IGHV --regex --outname heavy_only --outdir /home/phil/wt/results_032622_pm/

FileExistsError: [Errno 17] File exists: '/home/phil/wt/results_032622_pm'

In [ ]:
# Assign clonal groups
!DefineClones.py -d results_032622_pm/heavy_productive_parse-select.tsv --vf v_call \
--model ham --norm len --dist 0.01 --format airr --nproc 8 \
--outname heavy_productive_cloned --outdir results_032622_pm/

In [ ]:
# Reconstructing VDJ germline sequences
# -r for IMGT references used previously
# -g dmask removes D segment call for BCR, since it often has low confidence
# --cloned for generating a single germline consensus length for each clone
!CreateGermlines.py -d results_032622_pm/heavy_productive_cloned_clone-pass.tsv -g dmask --cloned -r /usr/local/share/germlines/imgt/mouse/vdj/imgt_mouse_IGHV.fasta /usr/local/share/germlines/imgt/mouse/vdj/imgt_mouse_IGHD.fasta /usr/local/share/germlines/imgt/mouse/vdj/imgt_mouse_IGHJ.fasta

In [6]:
!BuildTrees.py -d changeo_10x_output_032822/filtered_contig_heavy_germ-pass.tsv --minseq 2 --clean all \
    --igphyml --collapse --nproc 8 --asr 0.9

   START> BuildTrees
    FILE> filtered_contig_heavy_germ-pass.tsv
COLLAPSE> True

PROGRESS> 14:38:46 |Done                                              | 0.0 min

        OUTPUT> filtered_contig_heavy_germ-pass_lineages.tsv
       RECORDS> 4474
INITIAL_FILTER> 28
          PASS> 6
          FAIL> 22
 NONFUNCTIONAL> 0
FRAMESHIFT_DEL> 0
FRAMESHIFT_INS> 0
 CLONETOOSMALL> 10
  CDRFWR_ERROR> 0
  GERMLINE_PTC> 0
    OTHER_FAIL> 0
     DUPLICATE> 12
           END> BuildTrees

START> IgPhyML GY94 tree estimation

    START> IgPhyML HLP analysis
 OPTIMIZE> lr
    TS/TV> e
wFWR,wCDR> e,e
   MOTIFS> WRC_2:0,GYW_0:1,WA_1:2,TW_0:3,SYC_2:4,GRS_0:5
  HOTNESS> e,e,e,e,e,e
    NPROC> 8

       OUTPUT> changeo_10x_output_032822/filtered_contig_heavy_germ-pass_igphyml-pass.tab
  TREE_LENGTH> 0.08
        LHOOD> -208.18
    KAPPA_MLE> 3.24
OMEGA_FWR_MLE> 0.67
OMEGA_CDR_MLE> 0.25
    WRC_2_MLE> 1.29
    GYW_0_MLE> 3.96
     WA_1_MLE> 0.64
     TW_0_MLE> -0.99
    SYC_2_MLE> -0.99
    GRS_0_MLE> -0.99

ST

In [7]:
!BuildTrees.py -d changeo_10x_output_032822/filtered_contig_light_productive-T.tsv --minseq 2 --clean all \
    --igphyml --collapse --nproc 8 --asr 0.9

   START> BuildTrees
    FILE> filtered_contig_light_productive-T.tsv
COLLAPSE> True

PROGRESS> 15:35:58 |Correcting frames and indels of sequences         | 0.0 minERROR> Cannot export datasets until sequences are clustered into clones.

